# Equalizer Comparison
This example uses discrete-channel model

In [ ]:
#------------------------------------------------------------------------------#
# Adding comm module path
#------------------------------------------------------------------------------#
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
#------------------------------------------------------------------------------#
# Import Libraries
#------------------------------------------------------------------------------#
import numpy as np
import matplotlib.pyplot as plt

pi = np.pi
from numpy import exp as exp
from numpy import log as log
from numpy import cos as cos
from numpy import sin as sin
from numpy import sqrt as sqrt
from numpy import sign as sign
from numpy import real as real
from numpy import imag as imag

from scipy import signal
from scipy.signal import convolve
from scipy.signal import lfilter
from scipy.linalg import toeplitz

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from utils.utils_filter import *
from utils.utils_waveform import *
from utils.utils_report import *

from adaptive_filters.lms import *
from adaptive_filters.rls import *

from modulation.bpsk_modulator import BpskModulator
from modulation.bpsk_demodulator import BpskDemodulator

## Generating Dataset

In [ ]:
#------------------------------------------------------------------------------#
# Data Parameters
#------------------------------------------------------------------------------#
N = 1000 # Number of bits to generate
bits = np.random.normal(0, 1, N) > 0
symbols = 2*bits - 1

#------------------------------------------------------------------------------#
# Channel Parameters
#------------------------------------------------------------------------------#
#c = np.array([0.90, -0.15, 0.20, 0.10, -0.05]) # G. Stüber's example 7.2 in Section 7, Principles of Mobile Communication
c = np.array([0.2835, 0.2031, 0.1455, 0.1043, 0.0747, 0.0535, 0.0384, 0.0275, 0.0197, 0.0141, 0.0101, 0.0072, 0.0052, 0.0037, 0.0027, 0.0019])
snr_db = 10
snr = 10**(snr_db/10)

#------------------------------------------------------------------------------#
# Received signal
#------------------------------------------------------------------------------#
# TODO: Check SNR
symbolsIsi = lfilter(c, 1, symbols)
s_p = power(symbols) # Signal power
n_p = s_p/snr # Noise power
noise = sqrt(n_p) * np.random.normal(0, 1, symbolsIsi.size)
symbolsIsi = symbolsIsi.astype(float) + noise

# Plotting
plt.figure()
plt.title('Interfered symbols')
plt.stem(symbolsIsi, basefmt=" ", linefmt="blue", markerfmt="bo")
plt.grid()
plt.xlabel('Symbol index')
plt.ylabel('c')
plt.show()

## Linear Equalizers

In [ ]:
# Common parameters

### Zero-Forcing

In [ ]:
# TODO

### Wiener

In [ ]:
# TODO

## Adaptive Linear Equalizers
Note: Training dataset can be chosen with different offset which corresponds delay. It changes EVM.

In [ ]:
# Parameters
Ne = c.size # Filter taps, It could be less

# LMS parameter
muLms = 0.1 # Convergence rate

# NLMS parameter
muNlms = 0.2 # Convergence rage

lmbdaRls = 0.999 # Forgetting factor
deltaRls = 0.1 # Regularization term 

### LMS Algorithm

In [ ]:
lms = Lms(Ne, muLms)

# Memory allocation to record evolution of adaptive filter
wRecLms = np.zeros((Ne, N-Ne))
evmLms = np.zeros(N-Ne)

# Using the training data to estimate the equalizer weights
for k in range(Ne,N):
    xk = symbolsIsi[(k-Ne):(k)]
    w, e = lms.evalChunk(xk, symbols[k-1])    
    
    # Record and equalize for each weight to monitor EVM
    y = filterB(w, symbolsIsi)
    wRecLms[:, k-Ne] = w.flatten()
    evmLms[k-Ne] = errorVectorMagnitude(y, symbols)

print("LMS Equalizer")
linkReport(y, symbols, BpskDemodulator())

# Plot EVM change
plt.figure()
plt.plot(evmLms)
plt.grid()
plt.title('LMS Equalizer with ' + str(N) + ' training bits and mu: ' + str(muLms))
plt.xlabel('Training sample number')
plt.ylabel('EVM')

# Plot filter taps
plt.figure()
plt.plot(wRecLms.T)
plt.grid()
plt.title('Evolution of LMS equalizer weights')
plt.xlabel('Training sample number')
plt.ylabel('Weight value')
plt.show()

### NLMS Algorithm

In [ ]:
nlms = Lms(Ne, muNlms, "nlms")

# Memory allocation to record evolution of adaptive filter
wRecNlms = np.zeros((Ne, N-Ne))
evmNlms = np.zeros(N-Ne)

# Using the training data to estimate the equalizer weights
for k in range(Ne,N):
    xk = symbolsIsi[(k-Ne):(k)]
    w, e = nlms.evalChunk(xk, symbols[k-1])    
    
    # Record and equalize for each weight to monitor EVM
    y = filterB(w, symbolsIsi)
    wRecNlms[:, k-Ne] = w.flatten()
    evmNlms[k-Ne] = errorVectorMagnitude(y, symbols)

print("NLMS Equalizer")
linkReport(y, symbols, BpskDemodulator())

# Plot EVM change
plt.figure()
plt.plot(evmNlms)
plt.grid()
plt.title('NLMS Equalizer with ' + str(N) + ' training bits and mu: ' + str(muNlms))
plt.xlabel('Training sample number')
plt.ylabel('EVM')

# Plot filter taps
plt.figure()
plt.plot(wRecNlms.T)
plt.grid()
plt.title('Evolution of NLMS equalizer weights')
plt.xlabel('Training sample number')
plt.ylabel('Weight value')
plt.show()

### RLS Algorithm

In [ ]:
rls = Rls(Ne, lmbdaRls, deltaRls)

# Memory allocation to record evolution of adaptive filter
wRecRls = np.zeros((Ne, N-Ne))
evmRls = np.zeros(N-Ne)

# Using the training data to estimate the equalizer weights
for k in range(Ne,N):
    xk = symbolsIsi[(k-Ne):(k)]
    w, e = rls.evalChunk(xk, symbols[k-1])    
    
    # Record and equalize for each weight to monitor EVM
    y = filterB(w, symbolsIsi)
    wRecRls[:, k-Ne] = w.flatten()
    evmRls[k-Ne] = errorVectorMagnitude(y, symbols)

print("RLS Equalizer")
linkReport(y, symbols, BpskDemodulator())

# Plot EVM change
plt.figure()
plt.plot(evmRls)
plt.grid()
plt.title('RLS Equalizer with ' + str(N) + ' training bits,  lambda: ' + str(lmbdaRls) + ' and delta: ' + str(deltaRls))
plt.xlabel('Training sample number')
plt.ylabel('EVM')

# Plot filter taps
plt.figure()
plt.plot(wRecRls.T)
plt.grid()
plt.title('Evolution of RLS equalizer weights')
plt.xlabel('Training sample number')
plt.ylabel('Weight value')
plt.show()

## Machine Learning Methods
WARNING: Following codes are hobby-level. Don't take them serious

In [ ]:
#------------------------------------------------------------------------------#
# Machine Learning
#------------------------------------------------------------------------------#
cLen = c.size
labeledLen = N-cLen

# Allocate mems
xSet = np.zeros((labeledLen, cLen))
ySet = np.zeros((labeledLen, 1))

# Get x and y set
for k in range(cLen,N):
    xSet[k-cLen, :] = xk = symbolsIsi[(k-cLen):(k)]
    ySet[k-cLen] = symbols[k-1]
    
# Creating pandas dataframe from numpy array
dataset = pd.DataFrame(xSet)
dataset['Y'] = ySet

dataset.head()

In [ ]:
# Form train and test set
xTrain, xTest, yTrain, yTest = train_test_split(dataset.iloc[:,:cLen], dataset['Y'], test_size=0.33, random_state=42)
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)

In [ ]:
# Random Forest Classifier
model = RandomForestClassifier()
model.fit(xTrain, yTrain)

# Predict from model
yPred = model.predict(xTest)

# Results
linkReport(yPred, yTest.to_numpy(), BpskDemodulator())
print('SNR: ' + str(snr_db) + ' [db]\n\tSER: ' + str(1-accuracy_score(yTest, yPred)))
print('\nClassification Report:')
print(classification_report(yTest, yPred))

In [ ]:
# LSTM
model_lstm = Sequential()

# Alternative-1
model_lstm.add(LSTM(1,
    activation="tanh",
    recurrent_activation="tanh",
    batch_input_shape=(None, cLen, 1), 
    return_sequences=False))

#Alternative-2
#model_lstm.add(LSTM(1, batch_input_shape=(None, Ne+1, 1), return_sequences=True))
#model_lstm.add(LSTM(1, return_sequences=False))

opt = keras.optimizers.Adam(learning_rate=0.1)
model_lstm.compile(loss='mean_absolute_error',
    optimizer=opt,
    metrics=['accuracy'])
model_lstm.summary()

In [ ]:
# Reshape to 3d for ltsm
n_train = 200
n_test = 200

_x_train = xSet[0:n_train, :]
_x_test = xSet[n_train:n_train+n_test+1, :]
_x_remain = xSet[n_train+n_test+1::, :]

_y_train = ySet[0:n_train]
_y_test = ySet[n_train:n_train+n_test+1]
_y_remain = ySet[n_train+n_test+1::]

_x_train = _x_train.reshape(_x_train.shape[0], _x_train.shape[1], 1)
_y_train = _y_train.reshape(_y_train.shape[0], _y_train.shape[1], 1)

_x_test = _x_test.reshape(_x_test.shape[0], _x_test.shape[1], 1)
_y_test = _y_test.reshape(_y_test.shape[0], _y_test.shape[1], 1)

_x_remain = _x_remain.reshape(_x_remain.shape[0], _x_remain.shape[1], 1)
_y_remain = _y_remain.reshape(_y_remain.shape[0], _y_remain.shape[1], 1)

In [ ]:
# Train model
history_ltsm = model_lstm.fit(_x_train, _y_train, epochs=10, validation_data=(_x_test, _y_test))

In [ ]:
results_lstm = model_lstm.predict(_x_remain)

len = 50 #_y_test.shape[0]
plt.scatter(range(len), results_lstm[0:len].reshape(len), c='r')
plt.scatter(range(len), _y_remain[0:len].reshape(len), c='g')

print("LSTM Equalizer")
linkReport(results_lstm, _y_remain, BpskDemodulator())

In [ ]:
plt.plot(history_ltsm.history['loss'])
plt.title('loss')
plt.show()
plt.plot(history_ltsm.history['val_loss'])
plt.title('val_loss')
plt.show()
#plt.plot(history_ltsm.history['acc'])
#plt.show()
#plt.plot(history_ltsm.history['val_acc'])
#plt.show()